<a href="https://colab.research.google.com/github/DrSubbiah/0.Linear_Algebra/blob/main/LinDep_Theory_Comput_LA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color="maroon"> **Reasoning vs Precision**

Human theoretical reasoning may sometimes differ from the algorithmic approaches used by computers. This is neither new nor unusual; rather, it is natural for machines, which are not able (or not designed) to understand symbolic forms and exact real numbers in the way humans do. Therefore, human reasoning and machine floating-point precision need not coincide. For further reading on this topic, see the *Deep Learning* (DL) book by Goodfellow et al.:
[https://www.deeplearningbook.org/contents/numerical.html](https://www.deeplearningbook.org/contents/numerical.html)

These notes attempt to exemplify one such situation.

Let us consider $X_1$ and $Y$, both randomly generated vectors of length $n$. Let $X_2 = 2 \times X_1$ (or simply $X_2 = 2X_1$), and let $X$ be a matrix whose columns are $X_1$ and $X_2$.

Human factual (theoretical) reasoning immediately leads to the following:

* $X^TX$ is a singular matrix
* At least one eigenvalue must be zero
* A warning not to proceed if one attempts to fit a linear model $Y \sim X_1 + X_2$

All of these (and similar other conclusions) are immediate for human reasoning. A human decision might be to drop a variable or revise the model. However, machine floating-point computations behave differently, and their behavior largely depends on how the algorithms are **calibrated**.

Below are examples from standard (and widely used) Python libraries to examine this behavior. In this exercise, another observation emerges: differences in results can arise due to ***algorithmic calibration***, which itself can be an important learning point.

## <font color="darkblue"> Details of Examples

* Five different random generators are used for $X_1$ and $Y$
* Two different libraries are used for fitting a linear model
* Results are compared
* A linear model is then fitted using only one dataset generated from a sampler
* Discrete uniform generators are used to retain expected similarity between human reasoning and machine computation, since floating-point issues do not arise when $X_1$ consists of integers

The conclusions from the summary tables are self-explanatory.

Finally, here is an exact line from the DL book:

***“Developers of low-level libraries should keep numerical issues in mind when implementing deep learning algorithms.”***



# <font color="maroon"> Example 1

## <font color="blue"> Necessary Libraries

In [ ]:
# ============================================================
# STEP 0: Imports
# ============================================================
import numpy as np
import pandas as pd
from scipy.stats import norm
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
import torch
import tensorflow as tf


## <font color="blue"> Fit the model and summarize the comparative results

In [ ]:

# ------------------------------------------------------------
# Initial set up
# ------------------------------------------------------------
N = 50
SEED = 123

results = []

# ------------------------------------------------------------
# Fit LM with two libraries
# ------------------------------------------------------------
def fit_statsmodels(Y, X1, X2):
    X = sm.add_constant(np.column_stack([X1, X2]))
    m = sm.OLS(Y, X).fit()
    return m.params[0], m.params[1], m.params[2]

def fit_sklearn(Y, X1, X2):
    X = np.column_stack([X1, X2])
    m = LinearRegression(fit_intercept=True).fit(X, Y)
    return m.intercept_, m.coef_[0], m.coef_[1]

def store(gen_lib, fit_lib, params):
    results.append({
        "data_gen_lib": gen_lib,
        "fit_lib": fit_lib,
        "intercept": params[0],
        "beta_X1": params[1],
        "beta_X2": params[2]
    })

# ------------------------------------------------------------
# Random number from NUMPY and fit OLS using two libraries
# ------------------------------------------------------------
np.random.seed(SEED)
X1 = np.random.normal(10, 5, N)
X2 = 2 * X1
Y  = np.random.normal(0, 4, N)

store("NumPy", "statsmodels", fit_statsmodels(Y, X1, X2))
store("NumPy", "sklearn",     fit_sklearn(Y, X1, X2))

# ------------------------------------------------------------
# Random number from SCIPY and fit OLS using two libraries
# ------------------------------------------------------------
np.random.seed(SEED)
X1 = norm.rvs(10, 5, N)
X2 = 2 * X1
Y  = norm.rvs(0, 4, N)

store("SciPy", "statsmodels", fit_statsmodels(Y, X1, X2))
store("SciPy", "sklearn",     fit_sklearn(Y, X1, X2))

# ------------------------------------------------------------
# Random number from PANDAS and fit OLS using two libraries
# ------------------------------------------------------------
np.random.seed(SEED)
df = pd.DataFrame({"X1": np.random.normal(10, 5, N)})
df["X2"] = 2 * df["X1"]
df["Y"]  = np.random.normal(0, 4, N)

store("Pandas", "statsmodels", fit_statsmodels(df["Y"], df["X1"], df["X2"]))
store("Pandas", "sklearn",     fit_sklearn(df["Y"], df["X1"], df["X2"]))

# ------------------------------------------------------------
# Random number from PYTORCH and fit OLS using two libraries
# ------------------------------------------------------------
torch.manual_seed(SEED)
X1 = torch.normal(10.0, 5.0, size=(N,))
X2 = 2 * X1
Y  = torch.normal(0.0, 4.0, size=(N,))

store("PyTorch", "statsmodels", fit_statsmodels(Y.numpy(), X1.numpy(), X2.numpy()))
store("PyTorch", "sklearn",     fit_sklearn(Y.numpy(), X1.numpy(), X2.numpy()))

# ------------------------------------------------------------
# Random number from TENSORFLOW and fit OLS using two libraries
# ------------------------------------------------------------
tf.random.set_seed(SEED)
X1 = tf.random.normal((N,), mean=10.0, stddev=5.0)
X2 = 2 * X1
Y  = tf.random.normal((N,), mean=0.0, stddev=4.0)

store("TensorFlow", "statsmodels", fit_statsmodels(Y.numpy(), X1.numpy(), X2.numpy()))
store("TensorFlow", "sklearn",     fit_sklearn(Y.numpy(), X1.numpy(), X2.numpy()))

# ------------------------------------------------------------
# FINAL RESULTS TABLE
# ------------------------------------------------------------
results_df = pd.DataFrame(results)
print(results_df)


  data_gen_lib      fit_lib  intercept   beta_X1   beta_X2
0        NumPy  statsmodels   0.777252 -0.012188 -0.024377
1        NumPy      sklearn   0.777252 -0.012188 -0.024377
2        SciPy  statsmodels   0.777252 -0.012188 -0.024377
3        SciPy      sklearn   0.777252 -0.012188 -0.024377
4       Pandas  statsmodels   0.777252 -0.012188 -0.024377
5       Pandas      sklearn   0.777252 -0.012188 -0.024377
6      PyTorch  statsmodels  -1.741812  0.021809  0.043617
7      PyTorch      sklearn  -1.741812  0.021809  0.043617
8   TensorFlow  statsmodels  -0.482770  0.000956  0.001913
9   TensorFlow      sklearn  -0.482770  0.000956  0.001913


/tmp/ipython-input-3909263479.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return m.params[0], m.params[1], m.params[2]


# <font color="maroon"> Example 2

## <font color="blue"> ONE DATASET, MULTIPLE LM FITS

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

# ------------------------------------------------------------
# CONFIG
# ------------------------------------------------------------
N = 50
SEED = 123

# ------------------------------------------------------------
# DATA GENERATION (single generator: NumPy)
# ------------------------------------------------------------
np.random.seed(SEED)
X1 = np.random.normal(10, 5, N)
X2 = 2 * X1
Y  = np.random.normal(0, 4, N)

# ------------------------------------------------------------
# FIT: STATSMODELS
# ------------------------------------------------------------
X_sm = sm.add_constant(np.column_stack([X1, X2]))
sm_model = sm.OLS(Y, X_sm).fit()

# ------------------------------------------------------------
# FIT: SKLEARN
# ------------------------------------------------------------
X_sk = np.column_stack([X1, X2])
sk_model = LinearRegression(fit_intercept=True).fit(X_sk, Y)

# ------------------------------------------------------------
# CONSOLIDATED RESULTS DF
# ------------------------------------------------------------
results_df = pd.DataFrame([
    {
        "data_gen_lib": "NumPy",
        "fit_lib": "statsmodels",
        "intercept": sm_model.params[0],
        "beta_X1": sm_model.params[1],
        "beta_X2": sm_model.params[2]
    },
    {
        "data_gen_lib": "NumPy",
        "fit_lib": "sklearn",
        "intercept": sk_model.intercept_,
        "beta_X1": sk_model.coef_[0],
        "beta_X2": sk_model.coef_[1]
    }
])

print(results_df)


  data_gen_lib      fit_lib  intercept   beta_X1   beta_X2
0        NumPy  statsmodels   0.777252 -0.012188 -0.024377
1        NumPy      sklearn   0.777252 -0.012188 -0.024377


# <font color="maroon"> Example 3

## <font color="blue">  DISCRETE X1, SAME DATA, MULTIPLE LM FITS

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

# ------------------------------------------------------------
# CONFIG
# ------------------------------------------------------------
N = 50
SEED = 123

# ------------------------------------------------------------
# DATA GENERATION (Discrete Uniform)
# ------------------------------------------------------------
np.random.seed(SEED)

X1 = np.random.randint(1, 101, size=N)   # discrete uniform {1,...,100}
X2 = 2 * X1
Y  = np.random.normal(0, 4, size=N)

# ------------------------------------------------------------
# FIT: STATSMODELS
# ------------------------------------------------------------
X_sm = sm.add_constant(np.column_stack([X1, X2]))
sm_model = sm.OLS(Y, X_sm).fit()

# ------------------------------------------------------------
# FIT: SKLEARN
# ------------------------------------------------------------
X_sk = np.column_stack([X1, X2])
sk_model = LinearRegression(fit_intercept=True).fit(X_sk, Y)

# ------------------------------------------------------------
# CONSOLIDATED RESULTS DF
# ------------------------------------------------------------
results_df = pd.DataFrame([
    {
        "data_gen_lib": "NumPy (discrete uniform)",
        "fit_lib": "statsmodels",
        "intercept": sm_model.params[0],
        "beta_X1": sm_model.params[1],
        "beta_X2": sm_model.params[2]
    },
    {
        "data_gen_lib": "NumPy (discrete uniform)",
        "fit_lib": "sklearn",
        "intercept": sk_model.intercept_,
        "beta_X1": sk_model.coef_[0],
        "beta_X2": sk_model.coef_[1]
    }
])

print(results_df)


               data_gen_lib      fit_lib  intercept   beta_X1   beta_X2
0  NumPy (discrete uniform)  statsmodels   1.088742 -0.002019 -0.004037
1  NumPy (discrete uniform)      sklearn   1.088742 -0.002019 -0.004037


# <font color="maroon"> Example 4

## <font color="blue">  Misc - Matrix Computations

In [ ]:
import numpy as np

# ------------------------------------------------------------
# CONFIG
# ------------------------------------------------------------
N = 50
SEED = 123

np.random.seed(SEED)

# ------------------------------------------------------------
# DATA (EXACT LD)
# ------------------------------------------------------------
X1 = np.random.normal(10, 5, N)
X2 = 2 * X1

X = np.column_stack([X1, X2])

# ------------------------------------------------------------
# COMPUTE X'X
# ------------------------------------------------------------
XTX = X.T @ X

# ------------------------------------------------------------
# DETERMINANT AND EIGENVALUES
# ------------------------------------------------------------
det_XTX = np.linalg.det(XTX)
eigvals_XTX = np.linalg.eigvals(XTX)

# ------------------------------------------------------------
# OUTPUT
# ------------------------------------------------------------
print("X'X matrix:\n", XTX)
print("\ndet(X'X):", det_XTX)
print("\nEigenvalues of X'X:", eigvals_XTX)


X'X matrix:
 [[ 6835.58041892 13671.16083785]
 [13671.16083785 27342.3216757 ]]

det(X'X): 0.0

Eigenvalues of X'X: [-3.63797881e-12  3.41779021e+04]


# <font color="maroon"> **Notes to Leaders in Data Science / Machine Learning**

Leaders themselves don’t code in C or assembly, they should care about low-level calibration because:

- Numerical stability matters: small floating-point differences can cause models to fail, produce NaNs, or give inconsistent results across platforms.

- Cross-library consistency: different low-level implementations (BLAS, LAPACK, MKL, GPU kernels) can produce slightly different outputs.

- Edge cases and safety: in production, edge cases may break high-level assumptions.

- Efficient resource usage: low-level optimization affects speed and memory.

So a good leader in DS/ML should:

- Ensure the team understands low-level calibration and precision limits.

- Encourage testing across platforms and libraries.

- Don’t blindly trust high-level outputs, especially in critical applications

---

1. **Understand numerical foundations**

   * Know that high-level frameworks rely on low-level libraries (C/Fortran/GPU kernels) and floating-point arithmetic.
   * Encourage awareness of **precision, rounding errors, and numerical stability**.

2. **Promote cross-platform testing**

   * Ensure models and pipelines are tested across libraries, platforms, and devices to detect subtle inconsistencies.

3. **Encourage edge-case exploration**

   * Teams should identify situations where floating-point limits or ill-conditioned matrices can break assumptions.

4. **Foster communication between levels**

   * High-level developers and low-level engineers should collaborate: optimizations, calibration, and fixes in low-level code must inform high-level design.

5. **Prioritize reproducibility and auditability**

   * Document random seeds, library versions, and computation details to track where differences arise.

6. **Balance trust and verification**

   * Don’t blindly trust high-level outputs — cultivate a culture of **reasoning over precision**, i.e., **understand why a result occurs, not just that it occurs**.